<a href="https://colab.research.google.com/github/lyj7518/CP1/blob/main/preprocessing_%EC%9C%A0%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기
- web_types : 0=당근마켓, 1=번개장터

In [78]:
import pandas as pd

In [79]:
df = pd.read_csv('/content/drive/MyDrive/CodeStates Project 1/데이터/concat_df.csv', index_col = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28605 entries, 0 to 8854
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      28605 non-null  object 
 1   price      28605 non-null  object 
 2   like       28605 non-null  int64  
 3   view       28605 non-null  float64
 4   category   28605 non-null  object 
 5   article    28605 non-null  object 
 6   location   28605 non-null  object 
 7   time       28605 non-null  object 
 8   web_types  28605 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 2.2+ MB


# 전처리 필요 항목
- 당근마켓 : manner degree, chat 지우기
- title : "구해요", "삽니다" , "구매합니다", "스트랩" , "밴드", "충전", "루프", "스트렙" 제거
- price : 쉼표 제거
- category : 
    - 당근마켓 데이터 : 디지털 기기 빼고 제거
    - 번개장터 데이터 : 웨어러블(워치) 빼고 제거
- location : 당근마켓 데이터랑 지역 단위 맞추기, "전국" ➡ nan 값으로 바꾸기
- article
    - "\n" 제거
    - "미사용", "미개봉", "새 세품", "미사용", "새상품" ➡ 미개봉 column 만들어서 1 값 부여
- 제품별 정가 (판매가) 컬럼 추가
- 할인율 컬럼 추가

## 전처리

### 1) price 쉼표 제거 및 형변환
- price : 쉼표 제거하고 int형으로 형변환
- view : float ➡ int형으로 형변환
- time : object ➡ float형으로 형변환

In [80]:
from numpy import NaN

# price 쉼표 제거 & 형변환
def del_rest(df, col):
    df[col] = df[col].str.replace(',', '').astype('int64')

df['view'] = df['view'].astype('int64')
df['time'] = df['time'].replace('?', NaN).astype('float64')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28605 entries, 0 to 8854
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      28605 non-null  object 
 1   price      28605 non-null  object 
 2   like       28605 non-null  int64  
 3   view       28605 non-null  int64  
 4   category   28605 non-null  object 
 5   article    28605 non-null  object 
 6   location   28605 non-null  object 
 7   time       19750 non-null  float64
 8   web_types  28605 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 2.2+ MB


### 2) location 단위 맞추기
- ex) 서울시 강남구 ➡ 서울

In [82]:
# 수정 전 location
df.tail(10)

,title,price,like,view,category,article,location,time,web_types
8845,갤럭시워치4 핑크골드40mm,"250,000",1,86,웨어러블(워치),구매후 두번착용 ..시계 안차서 판매합니다.\n구성품 다 있으며 .액정보호필름 부착...,서울특별시 송파구 삼전동,NaN,1
8846,"갤럭시 워치 4 40,44 블투 LET 미개봉 팜이다","200,000",1,34,스마트폰,미개봉품 입니다\n연락주세요,전국,NaN,1
8847,갤럭시워치3(41mm),"120,000",1,47,웨어러블(워치),한달정도 사용한 갤워치3 판매합니다!\n베젤링이 부착돤 상태입니다\n(물론 탈부착은...,광주광역시 남구 효덕동,NaN,1
8848,갤럭시워치 LG U+ SM-R805N,"60,000",3,103,웨어러블(워치),2019년에 삿어요\n중전기하고 충전독 2개 드립니다.\n화면은 살때 필름 그대로 ...,전국,NaN,1
8849,"갤럭시 워치4 클래식 46mm 블랙 팝니다. (블루투스, 미착용)","295,000",2,234,웨어러블(워치),"갤럭시 워치4 클래식 46mm 블랙\nSM-R890 (블루투스, wifi, GPS...",대구광역시 중구 대봉1동,NaN,1
8850,갤럭시워치41mm시계,"120,000",0,38,기타(시계),에누리 가능하구요 스트랩 드립니다\n,전국,NaN,1
8851,미개봉 갤럭시 워치 4 40mm 실버,"220,000",2,81,케이스/보호필름/액세서리,미개봉 새상품입니다\n택배 발송은 3월 14일 이후에 가능해요~~,광주광역시 서구 농성1동,NaN,1
8852,삼성 갤럭시 워치 액티브2 44 스테인리스,"110,000",1,180,웨어러블(워치),제목 그대로 입니다!! 줄 바꾸는 툴킷이 안보이네요 ㅠㅠ 그리고 좀 뻑뻑해영,경상남도 창원시 성산구 웅남동,NaN,1
8853,갤럭시 워치4 블랙 40mm,"230,000",0,88,웨어러블(워치),제조연월: 2021. 08.\nBluetooth_Wi-Fi_GPS\n\n갤럭시 워치...,경기도 화성시 팔탄면,NaN,1
8854,갤럭시워치 액티브2 풀박스,"90,000",1,107,웨어러블(워치),구성품:본체+충전기+박스+스트랩+강화필름\n작년에 구입하였습니다.,경기도 평택시 서정동,NaN,1


In [83]:
# location 단위 맞추는 함수
def lo(df):
    if '충청남도' in df:
        df = '충남'
    elif '충청북도' in df:
        df =  '충북'
    elif '경상남도' in df:
        df = '경남'
    elif '경상북도' in df:
        df ='경북'
    elif '전라남도' in df:
        df = '전남'
    elif '전라북도' in df:
        df ='전북'
    elif '강원도' in df:
        df = '강원도'
    elif '경기도' in df:
        df = '경기도'
    else:
        df = df[:2]
    return df

In [84]:
# 함수 적용
df['location'] = df['location'].apply(lo)

In [85]:
# 이상값 확인 -> 251개 row 제거
df['location'].value_counts()

경기도    7012
서울     6257
전국     2571
인천     1668
부산     1501
대구     1309
경남     1221
경북      887
충남      855
대전      822
광주      786
충북      697
전북      668
울산      620
강원도     542
전남      516
제주      226
세종      200
0        79
1        62
2        25
5        13
4        12
7         7
3         5
10        4
6         4
9         4
11        4
8         3
19        2
32        2
15        2
42        2
5,        2
17        2
애플        1
31        1
13        1
12        1
25        1
23        1
60        1
26        1
7,        1
64        1
00        1
35        1
27        1
Name: location, dtype: int64

In [86]:
# 이상값 제거
df = df[df["location"].str.contains("경기도|서울|전국|인천|부산|대구|경남|경북|충남|대전|광주|충북|전북|울산|강원도|전남|제주|세종")]

In [87]:
# 제거 후 location values
df['location'].value_counts()

경기도    7012
서울     6257
전국     2571
인천     1668
부산     1501
대구     1309
경남     1221
경북      887
충남      855
대전      822
광주      786
충북      697
전북      668
울산      620
강원도     542
전남      516
제주      226
세종      200
Name: location, dtype: int64

In [88]:
# 제거 후 shape
df.shape

(28358, 9)

### 3) article에서 미개봉 컬럼 추출
- "미사용", "미개봉", "새 세품", "미사용", "새상품" ➡ 미개봉 column 만들어서 1 값 부여

In [89]:
df['unused'] = df['article'].str.contains("미개봉 | 미사용 | 미착용 | 미개봉 | 새제품 | 새 제품 | 새상품 |새 상품").apply(lambda x : 1 if x==True else 0)

In [96]:
# 미개봉 제품 3693개 (전체의 13%)
df[df['unused']==1]

,title,price,like,view,category,article,location,time,web_types,unused
7,애플워치7 스타라이트 GPS 41mm,399000,0,23,디지털기기,\n이미 44를 사용중이라 보관 해두고있는 41은 판매하려고합니다 :) \n2022...,대구,0.019,0,1
8,애플워치 40mm 스트랩,6000,0,17,디지털기기,\n거의 새상품 입니다. 엄청 가벼워요’!\n,경북,0.020,0,1
9,애플워치se 44mm 미개봉 삽니당,300000,0,1,삽니다,\n워치 미개봉 삽니다 연락부탁드려요\n,충북,0.021,0,1
35,애플워치5 스테인리스 블랙 44미리 미개봉 팝니다,400000,5,266,디지털기기,\n애플워치5 스테인리스 블랙 44미리 미개봉 팝니다\nSE동일칩셋 + 셀룰러 + ...,서울,0.030,0,1
38,애플워치6 실버 40mm 스테인리스 판매합니다.,465000,0,9,디지털기기,\n애플워치6 실버 40mm 스테인리스 판매합니다.\n사진처럼 보증기간은 23년 3...,서울,0.022,0,1
...,...,...,...,...,...,...,...,...,...,...
8829,갤럭시워치3 블루투스 45mm A급 제품,"140,000",1,365,웨어러블(워치),갤럭시워치3 블루투스 45mm A급 제품\n\n구성품: 사진상본체+충전기\n\n커버...,대구,NaN,1,1
8837,미개봉 새제품] 갤럭시 워치4 클래식 46 LTE,"340,000",0,64,생활가전,[미개봉 새제품] 갤럭시 워치4 클래식 46 LTE \n실버색상 자급제\n\n블랙보...,서울,NaN,1,1
8844,갤럭시워치 4 46mm,"300,000",1,54,웨어러블(워치),구매하고 사용하지 않아 판매합니다 \n미개봉 상태고요 편하게 연락주시면 감사드리겠습니다,전국,NaN,1,1
8849,"갤럭시 워치4 클래식 46mm 블랙 팝니다. (블루투스, 미착용)","295,000",2,234,웨어러블(워치),"갤럭시 워치4 클래식 46mm 블랙\nSM-R890 (블루투스, wifi, GPS...",대구,NaN,1,1
